In [4]:
valid_amount = []
def _amount_():

    '''promt user for amount'''

    try:
        flag = True
        amount = input()

        '''checking for valid input'''
        for a in amount:
            if a not in str(list(range(0,10))):
                flag = False
                raise TypeError('Amount must be numerical')
        if flag:
            valid_amount.append(float(amount))  

        '''find invalid input''' 
            
    except TypeError:
        _amount_()


In [8]:
valid_name = []
def name_lookup():

    '''prompt user to put in only valid customer name and account type'''

    try:
        account_name = input('Customer name:')
        account_type = input('Checking or Savings or Credit')
        if account_name not in customer_collection.keys() or account_type not in ['Savings','savings','checking','Checking','credit',"Credit"]:
            raise ValueError('invalid name/account type')
        else:
            valid_name.append((account_name,account_type))
    except ValueError:
        name_lookup()

class Account:
    def __init__(self, account_id,balance, interest = 0):
        
        '''checking if account id is string'''

        if not isinstance(account_id,str):
            raise TypeError('Account ID must be string')
        if not isinstance(balance,str):
            raise TypeError('Balance must be string')    
        for id in account_id:
            if id not in str(list(range(0,10))):
                raise ValueError('Account ID must be within 0-9')
        for num in balance:
            if num not in str(list(range(0,10))) and num != '.':
                raise ValueError('Balance must be numerical')
        
        '''assign the attributes'''

        self.account_id = account_id
        self.balance = float(balance)
        self.interest = interest

    '''print message'''

    def __str__(self):
        return f'{self.account_id},{round(self.balance,2)},{self.interest}'

    def _withdraw_(self):
        
        print('Amount: ',flush = True)
        _amount_()
        deducted_amount = valid_amount[-1]

        '''prevent user from withdrawing more than their balance '''

        self.balance -=  deducted_amount
        if self.balance < 0:
            print('Cannot proceed your withdrawal')
            self.balance += deducted_amount
    
    def _deposit_(self):
        
        print('Amount: ',flush = True)
        _amount_()
        added_amount = valid_amount[-1]
        self.balance += added_amount
        
    
class Checking(Account):

    '''inherit account class and force interest to be 0'''
    def __init__(self,account_id,balance,interest):
        super().__init__(account_id,balance,interest)
        self.interest = 0
    
    '''return message'''

    def __str__(self):
        return f'{self.account_id}  {round(float(self.balance),2)}'

class Savings(Account):

    '''inherit account class and force interest to be 1'''

    def __init__(self,account_id,balance,interest):
        super().__init__(account_id,balance,interest)
        self.interest = 1
    
    '''return message'''

    def __str__(self):
        return f'{self.account_id} {round(float(self.balance),2)}'
    
class Credit(Account):

    def __init__(self,account_id,balance,interest,credit_limit):
        super().__init__(account_id,balance)
        self.interest = interest
        self.credit_limit = credit_limit

    def __str__(self):
        return f"{self.account_id} {self.credit_limit}"
    
    def __credit_card_charge__(self):

        '''promt user to choose amount'''

        print('Amount: ', flush = True)
        _amount_()
        increased_amount = valid_amount[-1]

        '''prevent user to charge more than their limit'''
            
        self.balance += increased_amount
        if increased_amount > self.credit_limit:
            print('You have passed your credit limit')
            self.balance -= increased_amount

    def credit_card_payment(self):
        
        account_name,account_type = valid_name[-1]
        print("Amount: ", flush = True)
        _amount_()
        amt = valid_amount[-1]
        
        if amt > customer_collection[account_name].savings.balance or amt > customer_collection[account_name].checking.balance:
            return f"Inputted amount higher than balance in {account_type}"
        elif amt > self.balance:
            return f"Inputted amount higher than outstanding balance"
        else:
            if account_type in ['Savings','savings']:
                customer_collection[account_name].savings.balance -= amt
                self.balance -= amt
            elif account_type in ['Checking',"Checking"]:
                customer_collection[account_name].savings.balance -= amt
                self.balance -= amt
                
        
        self.balance -= amt
        customer_collection[account_name].balance -= amt

class Customer:
    def __init__(self,account_id,checking,savings,credit):

        self.account_id = account_id

        self.checking = checking

        self.savings = savings

        self.credit = credit

    def __str__(self):
        return f"{self.account_id,self.checking, self.savings,self.credit}"

import csv

'''for storing newly created Customer object'''
customer_collection = {}

def import_account():
    global customer_collection
    step1 = input("which file do you want to access to?")
    with open(f'{step1}', 'r') as readfile:
        account = csv.DictReader(readfile)
        for row in account:
            checking_object = Checking(row['checking_id'], row['checking_balance'], 0)
            saving_balance = Savings(row['savings_id'], row['savings_balance'], 0)
            credit = Credit(row['credit_id'], row['credit_balance'], row['credit_limit'], 0)
            customer = Customer(row['username'], checking_object, saving_balance, credit)
            customer_collection[row['username']] = customer

def exit_interface():
    global customer_collection 
    paste_file = input("What file do you want to put your new changes? ")
    with open(f"{paste_file}", 'w') as readfile:
        writer = csv.writer(readfile)
        writer.writerow(['username','checking_id','checking_balance','savings_id','savings_balance','credit_id','credit_balance','credit_limit'])
        rows = []
        for keys in customer_collection.keys():
            str_customer = customer_collection[keys]
            rows.append(str(str_customer))
        writer.writerows(rows)
        
def interface():

    '''getting information'''

    import_account()

    '''display options and prompt user to choose'''

    print(' 1. View Customer\n',
            '2. Deposit\n',
            '3. Withdraw\n',
            '4. Credit Card Charge\n',
            '5. Credit Card Payment\n',
            '6. Exit\n')
    user_choice = input('Enter your 1/2/3/4/5/6')
    
    while user_choice != '6':

        '''preventing invalid choice'''

        while user_choice not in str(list(range(1,8))):
            user_choice = input('Enter your 1/2/3/4/5/6')

        if user_choice == '1':
            for i in customer_collection.values():
                print(i)
            
        elif user_choice == '2':
            name_lookup()
            account_name, account_type = valid_name[-1]

            if account_type in ['Checking','checking']:
                customer_collection[account_name].checking._deposit_()
            elif account_type in ['Savings','savings']:
                customer_collection[account_name].savings._deposit_()

            '''withdraw action'''

        elif user_choice == '3':

            '''prompt the user to choose customer name and account type'''
            '''avoid typo/invalid name/account'''
                        
            name_lookup()
            account_name,account_type = valid_name[-1]

                
            '''proceed withdrawing by calling withdraw method for corresponding account type'''

            if account_type in ['Checking','checking']:
                customer_collection[account_name].checking._withdraw_()
            elif account_type in ['Savings','savings']:
                customer_collection[account_name].savings._withdraw_()
                
            '''creditcard charge method'''
                
        elif user_choice == '4':
                
            '''prompt user to choose customer name and proceed the method'''
            name_lookup()
            account_name,account_type = valid_name[-1]
                
            '''perform credit card charge action'''

            customer_collection[account_name].credit.__credit_card_charge__()
        elif user_choice == '4':

            name_lookup()
            account_name,account_type = valid_name[-1]

            customer_collection[account_name].credit.__credit_card_charge__()

        if user_choice == '5':
            
            name_lookup()
            account_name,account_type = valid_name[-1]
            customer_collection[account_name].credit.credit_card_payment()

        if user_choice == '6': 

            exit_interface()

        print(' 1. View Customer\n',
            '2. Deposit\n',
            '3. Withdraw\n',
            '4. Credit Card Charge\n',
            '5. Credit Card Payment\n',
            '6. Exit\n')
        user_choice = input('Enter your 1/2/3/4/5/6')

interface()

{'username': 'amohr', 'checking_id': '1337', 'checking_balance': '43.00', 'savings_id': '0666', 'savings_balance': '101.45', 'credit_id': '1729', 'credit_balance': '5000.00', 'credit_limit': '5000'}
{'username': 'bbaggins', 'checking_id': '2890', 'checking_balance': '15345.49', 'savings_id': '2941', 'savings_balance': '15577483.00', 'credit_id': '3021', 'credit_balance': '0.00', 'credit_limit': '50000'}
{'username': 'emusk', 'checking_id': '0001', 'checking_balance': '21588737.58', 'savings_id': '0002', 'savings_balance': '1000000000.00', 'credit_id': '0003', 'credit_balance': '435678.58', 'credit_limit': '10000000'}
 1. View Customer
 2. Deposit
 3. Withdraw
 4. Credit Card Charge
 5. Credit Card Payment
 6. Exit

